In [1]:
import numpy as np
import opt_einsum as oe
from copy import deepcopy
import itertools
from scipy import integrate
import pickle as pk
import copy
import math
import scipy

import torch
torch.set_default_dtype(torch.float64)

import os
import sys
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/pyTN'))
sys.path.append(os.path.realpath('/Users/wei/Documents/physics/code/pyTN/pytn'))

import pytn

import sympy as sp
from sympy import pprint, simplify, expand, latex
from IPython.display import display
from fractions import Fraction
# sp.init_printing (use_latex="mathjax", latex_mode="equation")
sp.init_printing(use_latex='mathjax', latex_mode='equation', fontsize='20pt')

In [23]:
def Ising_exact_fe(K):
    r'''
    compute the exact free energy of 2D Ising model in the themodynamic limit
    https://arxiv.org/pdf/1706.02541.pdf
    '''

    def fun(x):
        k = 2.0*np.sinh(2.0*K)/(np.cosh(2.0*K))**2
        return np.log(1.0+np.sqrt(1.0-(k**2)*(np.cos(x))**2))

    return -0.5*np.log(2.0)-np.log(np.cosh(2.0*K))-(1.0/(2.0*np.pi))*integrate.quad(fun, 0.0, np.pi)[0]

def Ising_exact_fe_2(K):
    r'''
    https://en.wikipedia.org/wiki/Square_lattice_Ising_model
    '''

    def fun(x):
        k = 1.0/(np.sinh(2.0*K))**2
        return np.log(np.cosh(2.0*K)**2+(1.0/k)*np.sqrt(1.0+k**2-2.0*k*np.cos(2*x)))

    return -0.5*np.log(2.0)-(1.0/(2.0*np.pi))*integrate.quad(fun, 0.0, np.pi)[0]

K = 0.1
print(Ising_exact_fe(K), Ising_exact_fe_2(K))

-0.7032312422858326 -0.7032312422858324


In [25]:
b = sp.Symbol(r'\beta', real=True)
t = sp.Symbol(r'\theta', real=True)
J = sp.Symbol('J', real=True)

# sp.integrate, to compute the integral
# sp.Integral, just create a symbolic integral expression

k = 1/(sp.sinh(2*b*J))**2
e = -sp.log(2)/2-(1/(2*sp.pi))*sp.Integral(sp.log(sp.cosh(2*b*J)**2+(1/k)*sp.sqrt(1+k**2-2*k*sp.cos(2*t))), (t, 0, sp.pi))
# display(e)
# display(e.evalf(subs={b: 0.1, J:1.0}))

def F(b):
    k = 1/(sp.sinh(2*b*J))**2
    return -sp.log(2)/2-(1/(2*sp.pi))*sp.Integral(sp.log(sp.cosh(2*b*J)**2+(1/k)*sp.sqrt(1+k**2-2*k*sp.cos(2*t))), (t, 0, sp.pi))

beta = 0.5
display(F(b))
display(F(b).evalf(subs={b: beta, J:1.0}))
display(sp.diff(F(b), b))
display(sp.diff(F(b), b).evalf(subs={b: beta, J:1.0}))


  π                                                                           
  ⌠                                                                           
  ⎮    ⎛     ___________________________________________                      
  ⎮    ⎜    ╱   2⋅cos(2⋅\theta)               1              2                
  ⎮ log⎜   ╱  - ──────────────── + 1 + ──────────────── ⋅sinh (2⋅J⋅\beta) + co
  ⎮    ⎜  ╱         2                      4                                  
  ⎮    ⎝╲╱      sinh (2⋅J⋅\beta)       sinh (2⋅J⋅\beta)                       
  ⌡                                                                           
  0                                                                           
- ────────────────────────────────────────────────────────────────────────────
                                                   2⋅π                        

                                  
                                  
              ⎞                   
  2           ⎟          

-1.02579281269492

 π                                                                            
 ⌠                                                                            
 ⎮                                                                            
 ⎮                                                                            
 ⎮          ___________________________________________                       
 ⎮         ╱   2⋅cos(2⋅\theta)               1                                
 ⎮ 4⋅J⋅   ╱  - ──────────────── + 1 + ──────────────── ⋅sinh(2⋅J⋅\beta)⋅cosh(2
 ⎮       ╱         2                      4                                   
 ⎮     ╲╱      sinh (2⋅J⋅\beta)       sinh (2⋅J⋅\beta)                        
 ⎮                                                                            
 ⎮                                                                            
 ⎮                                                                            
-⎮ ─────────────────────────────────────────────────

-1.74556457531255

In [369]:
class SquareClassicalIsing(object):
    r'''
    square lattice of 2D Ising model
    '''

    def __init__(self, K: float):
        r'''

        Parameters
        ----------
        chi_c: int, the cut-off of TRG SVD
        K: K = 1/beta
        '''

        # inherit the lattice parameters from the finite TPS
        self._phys_dim = 2
        # K = 1/beta
        self._K = K

        # lattice constant
        # self._lattice_const = 2

        spin = [2*(0.5-j) for j in range(self._phys_dim)]
        # the local tensor for Ising model
        temp = torch.zeros([self._phys_dim]*4)
        r = range(self._phys_dim)
        for l, k, j, i in itertools.product(r, r, r, r):
            temp[i, j, k, l] = -self._K*(spin[i]*spin[j]+spin[j]*spin[k]+spin[k]*spin[l]+spin[l]*spin[i])

        self._loc_tensor = torch.exp(temp).cdouble()

    def mv_lu(self, mpo, mps, lfp):
        # bring mps to a left-canonical form
        qs = []
        temp = mps[0]
        q, r = pytn.linalg.tensor_qr(temp, group_dims=((0, 2), (1,)), qr_dims=(1, 0))
        qs.append(q)
        temp = pytn.contract('ab,bcd->acd', r, mps[1])
        q, r = pytn.linalg.tensor_qr(temp, group_dims=((0, 2), (1,)), qr_dims=(1, 0))
        qs.append(q)

        lfp = pytn.contract(
            'abcd,aef,bfgh,chij,dkj->egik',
            lfp, qs[0], mpo[2], mpo[0], qs[0].conj())
        lfp = pytn.contract(
            'abcd,aef,bfgh,chij,dkj->egik',
            lfp, qs[1], mpo[3], mpo[1], qs[1].conj())
        
        return lfp/lfp.norm()

    def mv_ru(self, mpo, mps, rfp):
        # bring mps to a right-canonical form
        qs = []
        temp = mps[1]
        q, l = pytn.linalg.tensor_qr(temp, group_dims=((1, 2), (0,)), qr_dims=(0, 1))
        qs.append(q)
        temp = pytn.contract('abc,bd->adc', mps[0], l)
        q, l = pytn.linalg.tensor_qr(temp, group_dims=((1, 2), (0,)), qr_dims=(0, 1))
        qs.append(q)

        rfp = pytn.contract(
            'abc,dcef,gfhi,jki,behk->adgj',
            qs[0], mpo[3], mpo[1], qs[0].conj(), rfp)
        rfp = pytn.contract(
            'abc,dcef,gfhi,jki,behk->adgj',
            qs[1], mpo[2], mpo[0], qs[1].conj(), rfp)
        
        return rfp/rfp.norm()

    def up_boundary_mps_solver(self, le, re, col_mpo, initial_tensor=None):
        r'''
        solve a MPS tensor, only work for 2*2 unit cell
        '''

        rho = le.shape[0]

        def _mv(v):

            t = torch.from_numpy(v.reshape(rho, rho, self._phys_dim)).cdouble()
            # --a     j--
            #      |e
            # --b--*--f--
            #      |g
            # --c--*--h--
            #      |i
            # --d     k--
            w = pytn.contract('abcd,befg,cghi,jfhk,aje->dki', le, col_mpo[0], col_mpo[1], re, t)

            return w.flatten().numpy()

        initial_v = None
        if initial_tensor is not None:
            initial_v = initial_tensor.flatten().numpy()
        dim_op = (rho**2)*self._phys_dim
        op = scipy.sparse.linalg.LinearOperator(shape=(dim_op, dim_op), matvec=_mv)
        vals, vecs = scipy.sparse.linalg.eigs(
            op, k=3, which='LM', v0=initial_v, maxiter=None,
            return_eigenvectors=True)
        inds = abs(vals).argsort()[::-1]
        sorted_vals, sorted_vecs = vals[inds], vecs[:, inds]
        # sorted_vals, sorted_vecs = vals, vecs

        return sorted_vals[0], torch.from_numpy(sorted_vecs[:, 0]).reshape(rho, rho, self._phys_dim)

    def up_boundary_mps_sweep(self, lfp, rfp, mpo, mps):

        

        # err = 1.0
        for i in range(4):
            mps[1], l = pytn.linalg.tensor_qr(mps[1], group_dims=((1, 2), (0,)), qr_dims=(0, 1))
            temp = pytn.contract('abc,bd->adc', mps[0], l)
            le = lfp
            re = pytn.contract('abc,dcef,gfhi,jki,behk->adgj', mps[1], mpo[3], mpo[1], mps[1].conj(), rfp)
            v0, mps[0] = self.up_boundary_mps_solver(le, re, col_mpo=(mpo[2], mpo[0]), initial_tensor=temp)

            mps[0], r = pytn.linalg.tensor_qr(mps[0], group_dims=((0, 2), (1,)), qr_dims=(1, 0))
            temp = pytn.contract('ab,bcd->acd', r, mps[1])
            le = pytn.contract('abcd,aef,bfgh,chij,dkj->egik', lfp, mps[0], mpo[2], mpo[0], mps[0].conj())
            re = rfp
            v1, mps[1] = self.up_boundary_mps_solver(le, re, col_mpo=(mpo[3], mpo[1]), initial_tensor=temp)

            # print('sweep', i, v0, v1)

        return (v0, v1), mps

    def vumps_22(self, rho: int):
        r'''
        2*2 Unit cell VUMPS
        '''
        
        def _boundary_mps_norm(mps):

            le = torch.eye(rho).cdouble()
            for i in range(2):
                le = pytn.contract('ab,acd,bed->ce', le, mps[i], mps[i].conj())

            return pytn.contract('aa', le)
        
        def _boundary_mps_cost(mpo, mps, lfp, rfp):

            lfp = pytn.contract(
                'abcd,aef,bfgh,chij,dkj->egik',
                lfp, mps[0], mpo[2], mpo[0], mps[0].conj())
            rfp = pytn.contract(
                'abc,dcef,gfhi,jki,behk->adgj',
                mps[1], mpo[3], mpo[1], mps[1].conj(), rfp)
            
            return pytn.contract('abcd,abcd', lfp, rfp)
    

        mpo = [self._loc_tensor]*4

        # randomly initialize up and down boundary MPS
        #    2
        #    |
        # 0--*--1
        mps = []
        for i in range(2):
            mps.append(torch.rand(rho, rho, self._phys_dim).cdouble())
        mps = torch.stack(mps, dim=0)
        mps_u, mps_d = mps.clone(), mps.clone()
        print(_boundary_mps_norm(mps))
        
        # initial left, right-fixed points  for up and down MPS
        fp_lu = torch.rand(rho, self._phys_dim, self._phys_dim, rho).cdouble()
        fp_ru = torch.rand(rho, self._phys_dim, self._phys_dim, rho).cdouble()
        fp_ld = torch.rand(rho, self._phys_dim, self._phys_dim, rho).cdouble()
        fp_rd = torch.rand(rho, self._phys_dim, self._phys_dim, rho).cdouble()
        fp_lu, fp_ru = fp_lu/fp_lu.norm(), fp_ru/fp_ru.norm()
        fp_ld, fp_rd = fp_ld/fp_ld.norm(), fp_rd/fp_rd.norm()

        err = 1.0
        for i in range(100):
            # power method to find a approximated fixed point
            # iterater for the left and right environments
            for j in range(40):
                old_fp_lu, old_fp_ru = fp_lu, fp_ru
                fp_lu, fp_ru = self.mv_lu(mpo, mps_u, fp_lu), self.mv_ru(mpo, mps_u, fp_ru)
                diff_lu, diff_ru = (old_fp_lu-fp_lu).norm(), (old_fp_ru-fp_ru).norm()
                if 0 == j % 100:
                    print('fixed points:', j, diff_lu, diff_ru)
            # solve the upper boundary MPS
            # sweep
            vs, mps_u = self.up_boundary_mps_sweep(fp_lu, fp_ru, mpo, mps_u)
            # print(_boundary_mps_norm(mps_u))
            if 0 == i % 10:
                print ('cost:', _boundary_mps_cost(mpo, mps_u, fp_lu, fp_ru), _boundary_mps_norm(mps_u), vs)

    def vumps_11(self, rho: int):

        def _mv_lu(mpo, mps, lfp):

            temp = pytn.contract('abc,ade,befg,chg->dfh', lfp, mps, mpo, mps.conj())

            return temp/temp.norm()

        def _mv_ru(mpo, mps, rfp):

            temp = pytn.contract('abc,dcef,ghf,beh->adg', mps, mpo, mps.conj(), rfp)

            return temp/temp.norm()

        def _up_boundary_mps_solver(le, re, mpo, initial_tensor=None):

            def _mv(v):

                t = torch.from_numpy(v.reshape(rho, rho, self._phys_dim)).cdouble()
                # --a     g--
                #      |d
                # --b--*--e--
                #      |f
                # --c     h--
                w = pytn.contract('abc,bdef,geh,agd->chf', le, mpo, re, t)

                return w.flatten().numpy()

            initial_v = None
            if initial_tensor is not None:
                initial_v = initial_tensor.flatten().numpy()
            dim_op = (rho**2)*self._phys_dim
            op = scipy.sparse.linalg.LinearOperator(shape=(dim_op, dim_op), matvec=_mv)
            vals, vecs = scipy.sparse.linalg.eigs(
                op, k=3, which='LM', v0=initial_v, maxiter=None,
                return_eigenvectors=True)
            inds = abs(vals).argsort()[::-1]
            sorted_vals, sorted_vecs = vals[inds], vecs[:, inds]
            # sorted_vals, sorted_vecs = vals, vecs

            return sorted_vals[0], torch.from_numpy(sorted_vecs[:, 0]).reshape(rho, rho, self._phys_dim)

        def _left_fp_solver(mps, mpo, initial_tensor=None):

            def _mv(v):

                t = torch.from_numpy(v.reshape(rho, self._phys_dim, rho)).cdouble()
                # --a--*--d
                #      |e
                # --b--*--f
                #      |g
                # --c--*--h
                w = pytn.contract('abc,ade,befg,chg->dfh', t, mps, mpo, mps.conj())

                return w.flatten().numpy()

            initial_v = None
            if initial_tensor is not None:
                initial_v = initial_tensor.flatten().numpy()
            dim_op = (rho**2)*self._phys_dim
            op = scipy.sparse.linalg.LinearOperator(shape=(dim_op, dim_op), matvec=_mv)
            vals, vecs = scipy.sparse.linalg.eigs(
                op, k=2, which='LM', v0=initial_v, maxiter=None,
                return_eigenvectors=True)
            inds = abs(vals).argsort()[::-1]
            sorted_vals, sorted_vecs = vals[inds], vecs[:, inds]
            # sorted_vals, sorted_vecs = vals, vecs

            return sorted_vals[0], torch.from_numpy(sorted_vecs[:, 0]).reshape(rho, self._phys_dim, rho)

        def _right_fp_solver(mps, mpo, initial_tensor=None):

            def _mv(v):

                t = torch.from_numpy(v.reshape(rho, self._phys_dim, rho)).cdouble()
                # a--*--b--
                #    |c
                # d--*--e--
                #    |f
                # g--*--h--
                w = pytn.contract('abc,dcef,ghf,beh->adg', mps, mpo, mps.conj(), t)

                return w.flatten().numpy()

            initial_v = None
            if initial_tensor is not None:
                initial_v = initial_tensor.flatten().numpy()
            dim_op = (rho**2)*self._phys_dim
            op = scipy.sparse.linalg.LinearOperator(shape=(dim_op, dim_op), matvec=_mv)
            vals, vecs = scipy.sparse.linalg.eigs(
                op, k=2, which='LM', v0=initial_v, maxiter=None,
                return_eigenvectors=True)
            inds = abs(vals).argsort()[::-1]
            sorted_vals, sorted_vecs = vals[inds], vecs[:, inds]
            # sorted_vals, sorted_vecs = vals, vecs

            return sorted_vals[0], torch.from_numpy(sorted_vecs[:, 0]).reshape(rho, self._phys_dim, rho)

        mpo = self._loc_tensor
        mps = torch.rand(rho, rho, self._phys_dim).cdouble()

        fp_lu = torch.rand(rho, self._phys_dim, rho).cdouble()
        fp_ru = torch.rand(rho, self._phys_dim, rho).cdouble()
        fp_ld = torch.rand(rho, self._phys_dim, rho).cdouble()
        fp_rd = torch.rand(rho, self._phys_dim, rho).cdouble()
        fp_lu, fp_ru = fp_lu/fp_lu.norm(), fp_ru/fp_ru.norm()
        fp_ld, fp_rd = fp_ld/fp_ld.norm(), fp_rd/fp_rd.norm()

        for i in range(200):
            # power method to find a approximated fixed point
            # iterater for the left and right environments
            '''
            for j in range(40):
                old_fp_lu, old_fp_ru = fp_lu, fp_ru
                mps_l, r = pytn.linalg.tensor_qr(mps, group_dims=((0, 2), (1,)), qr_dims=(1, 0))
                mps_r, l = pytn.linalg.tensor_qr(mps, group_dims=((1, 2), (0,)), qr_dims=(0, 1))
                fp_lu, fp_ru = _mv_lu(mpo, mps_l, fp_lu), _mv_ru(mpo, mps_r, fp_ru)
                diff_lu, diff_ru = (old_fp_lu-fp_lu).norm(), (old_fp_ru-fp_ru).norm()
                if 0 == j % 100:
                    print('fixed points:', j, diff_lu, diff_ru)
            '''
            mps_l, r = pytn.linalg.tensor_qr(mps, group_dims=((0, 2), (1,)), qr_dims=(1, 0))
            mps_r, l = pytn.linalg.tensor_qr(mps, group_dims=((1, 2), (0,)), qr_dims=(0, 1))
            v, fp_lu = _left_fp_solver(mps_l, mpo, initial_tensor=fp_lu)
            v, fp_ru = _right_fp_solver(mps_r, mpo, initial_tensor=fp_ru)
            # solve the upper boundary MPS
            val, mps = _up_boundary_mps_solver(fp_lu, fp_ru, mpo, initial_tensor=mps)
            # print(_boundary_mps_norm(mps_u))
            if 0 == i % 10:
                print (i, 'lambda:', val, abs(val))


def exact_free_energy(K):
    r'''
    compute the exact free energy of 2D Ising model in the themodynamic limit
    https://arxiv.org/pdf/1706.02541.pdf
    '''

    def fun(x):
        k = 2.0*np.sinh(2.0*K)/(np.cosh(2.0*K))**2
        return np.log(1.0+np.sqrt(1.0-(k**2)*(np.cos(x))**2))

    return -0.5*np.log(2.0)-np.log(np.cosh(2.0*K))-(1.0/(2.0*np.pi))*integrate.quad(fun, 0.0, np.pi)[0]

print(exact_free_energy(0.1))

-0.7032312422858326


In [370]:
ising = SquareClassicalIsing(K=0.5)
ising.vumps_11(rho=8)

f = exact_free_energy(K=0.5)
print(f)

0 lambda: (-2.032680650127191+2.812431323782701j) 3.4700951826132402
10 lambda: (-3.900821646010506+1.6209843781301345j) 4.2242158879638305
20 lambda: (-0.1941788833404683+4.467393959076173j) 4.471612038664092
30 lambda: (-2.9514968729858566-4.3955581125150145j) 5.294550491943754
40 lambda: (3.7039389015279376+2.758003974647792j) 4.617981085975234
50 lambda: (2.030420353307197-3.9256767070793206j) 4.419676958741358
60 lambda: (-3.3654523535845637+3.252017774216947j) 4.679945421484189
70 lambda: (3.8329044709853246+4.817245996414706j) 6.156055203916937
80 lambda: (-4.114879256368896+0.1778875208963208j) 4.1187225282344135
90 lambda: (-3.053272608429896-2.216453040606862j) 3.7729481447011306
100 lambda: (4.174146001029765+0.8730089791972391j) 4.264462394683741
110 lambda: (-3.669650491909129+0.634050669458368j) 3.7240240310998955
120 lambda: (-1.13340100432218-4.034388226407167j) 4.190571106421091
130 lambda: (-2.888093903362224+2.2446548794632952j) 3.6578083496181737
140 lambda: (3.8250

In [26]:
def wyx(x, y, z):

    return x+y+z

In [27]:
a, b, c = 1, 2, 3

res = wyx(a, b, c)
print(res)

6
